In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import folium

In [2]:
raw_wiki = pd.read_json('./WikiData/query.json')

In [3]:
raw_wiki.head()

,musical_group,musical_groupLabel,genre,genreLabel,country_of_origin,country_of_originLabel,location_of_formation,location_of_formationLabel
0,http://www.wikidata.org/entity/Q115283,Black River,http://www.wikidata.org/entity/Q38848,heavy metal,http://www.wikidata.org/entity/Q36,Poland,NaN,NaN
1,http://www.wikidata.org/entity/Q115697,Drottnar,http://www.wikidata.org/entity/Q970409,unblack metal,http://www.wikidata.org/entity/Q20,Norway,http://www.wikidata.org/entity/Q107135,Fredrikstad
2,http://www.wikidata.org/entity/Q115772,The Awesome Machine,http://www.wikidata.org/entity/Q83270,hard rock,http://www.wikidata.org/entity/Q34,Sweden,NaN,NaN
3,http://www.wikidata.org/entity/Q115910,My Silent Wake,http://www.wikidata.org/entity/Q186170,doom metal,http://www.wikidata.org/entity/Q145,United Kingdom,NaN,NaN
4,http://www.wikidata.org/entity/Q116227,Partenaire Particulier,http://www.wikidata.org/entity/Q187760,new wave,http://www.wikidata.org/entity/Q142,France,NaN,NaN


In [4]:
raw_wiki.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55779 entries, 0 to 55778
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   musical_group               55779 non-null  object
 1   musical_groupLabel          55779 non-null  object
 2   genre                       55779 non-null  object
 3   genreLabel                  55779 non-null  object
 4   country_of_origin           45838 non-null  object
 5   country_of_originLabel      45838 non-null  object
 6   location_of_formation       23564 non-null  object
 7   location_of_formationLabel  23564 non-null  object
dtypes: object(8)
memory usage: 3.4+ MB


In [5]:
df_w = raw_wiki.copy().drop(['musical_group','genre','country_of_origin','location_of_formation'], axis=1)

In [6]:
df_w = df_w[(raw_wiki['country_of_originLabel'].notna())|(raw_wiki['location_of_formationLabel'].notna())]

In [7]:
df_w['country_of_originLabel'].nunique()

209

In [8]:
df_w = df_w.drop_duplicates(['musical_groupLabel','genreLabel'])

df_w = df_w.rename(columns={'musical_groupLabel': 'artist',
                            'genreLabel': 'genre',
                            'country_of_originLabel': 'country_of_origin',
                            'location_of_formationLabel': 'location_of_formation'})
df_w.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47208 entries, 0 to 55778
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   artist                 47208 non-null  object
 1   genre                  47208 non-null  object
 2   country_of_origin      45063 non-null  object
 3   location_of_formation  23073 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [9]:
trebi_songs = pd.read_csv('./trebi/csv/songs.csv', delimiter=';')
# trebi_tags = pd.read_csv('./trebi/csv/tags.csv', delimiter=';')

In [10]:
trebi_songs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 579879 entries, 0 to 579878
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   spotify_id  579879 non-null  object 
 1   name        579878 non-null  object 
 2   artist      579878 non-null  object 
 3   position    579878 non-null  float64
 4   genre_name  579878 non-null  object 
dtypes: float64(1), object(4)
memory usage: 22.1+ MB


In [11]:
artists_unique = trebi_songs['artist'].dropna().unique()
artists_unique.sort()
len(artists_unique)

162833

In [12]:
print(artists_unique[::1628])

['!!!' 'AOA' 'Afraid of Heights' 'Alex Prospect, Becci' 'American Hi-Fi'
 'Anita Mui' 'Arkitech' 'Austin Webb' 'Banda De Cornetas Y Tambores'
 'Belén Moreno' 'Billy Chernoff'
 'Boaz van de Beatz, Mr. Polska, Ronnie Flex' 'Breezo Dolla' 'Byklubben'
 'Carlos David' 'Charles Dodge' 'Christian Gaubert' 'Coco de Clown'
 'Course Of Nature' 'DC Talk' 'Dag Spantell, Torry Enghs'
 'Datakult, DJ Ehm' 'Deep Ng' 'Die Atzen' 'Do Or Die' 'Dropout Year'
 'Eddie Holman' 'Elio Revé Y Su Orquesta' 'Eric Odeen' 'FF.C, Maria Rizou'
 'Fifth On The Floor' 'Francesco Baccini, Zero Plastica' 'Fushara'
 'George Botsford, Alan Fernie, Central Band Of The Royal Air Force, Wing Commander R E Wilkinson'
 'Glynis Johns' 'Grupo Cochichando' 'Hanak' 'Helmut Zacharias'
 'Hosein Behroozinia, Bijan Kamkar' 'Ignaz Schick, Martin Tétreault'
 'Isa GT, Lido Pimienta' 'Jakub Noha Band' 'Jeannie C. Riley'
 'Joaquin Clerch, Rafael Aguirre' 'Jon the Dentist, Ollie Jaye'
 "Julius Dietert's Band" 'Karl Seglem' 'Khoff' 'Kommunen' 

In [13]:
trebi_songs['artist'] = trebi_songs['artist'].dropna().apply(lambda string : string.split(', '))

In [14]:
trebi_songs = trebi_songs.explode('artist')

In [15]:
trebi_songs = trebi_songs.reset_index(drop=True)
trebi_songs

,spotify_id,name,artist,position,genre_name
0,6lV2MSQmRIkycDScNtrBXO,Airplanes (feat. Hayley Williams of Paramore),B.o.B,1.0,pop rap
1,6lV2MSQmRIkycDScNtrBXO,Airplanes (feat. Hayley Williams of Paramore),Hayley Williams,1.0,pop rap
2,4hrae8atte6cRlSC9a7VCO,Always On Time,Ja Rule,2.0,pop rap
3,4hrae8atte6cRlSC9a7VCO,Always On Time,Ashanti,2.0,pop rap
4,03tqyYWC9Um2ZqU0ZN849H,No Hands [feat. Roscoe Dash and Wale],Waka Flocka Flame,3.0,pop rap
...,...,...,...,...,...
715988,5QFexL3QF0DOBuxFFLxWQ7,Tool Time,Schmalspurtonbaumeister,196.0,deep deep tech house
715989,6VZtpaqYAq0IYT8miLyQGo,Arvensis,Tiramitsu,197.0,deep deep tech house
715990,0P2VdQmimKONv15Zk6vQ6o,I Give U Something 4 D Pain,Deep Matt,198.0,deep deep tech house
715991,5KRo9hqPlW1gtOZNH1eql6,Guns! - Original Mix,Nick Tonze,199.0,deep deep tech house


In [16]:
len(trebi_songs['artist'].unique())

177668

In [17]:
df_t = trebi_songs[['artist','genre_name']].drop_duplicates().sort_values('artist')[:-1]
df_t = df_t.rename(columns={'genre_name': 'genre'})

In [18]:
df = pd.merge(df_t,df_w,on='artist')
df = df.melt(id_vars=['artist', 'location_of_formation', 'country_of_origin'],
           value_vars=['genre_x','genre_y'],
           value_name='genre'
          ).drop('variable',axis=1)
df = df.drop_duplicates()
df = df[['artist','genre','location_of_formation','country_of_origin']]
df.nunique()

artist                   15419
genre                     1936
location_of_formation     2044
country_of_origin          138
dtype: int64

In [19]:
cities = pd.read_csv('~/Desktop/cities500.txt', sep='\t', header=None, usecols=[1,4,5,8,14,17,18], low_memory=False)
cities = cities.rename(columns={1:'cityName',4:'lat',5:'lon',8:'countryCode',14:'pop.',17:'region',18:'updatedLast'})
cities

,cityName,lat,lon,countryCode,pop.,region,updatedLast
0,Soldeu,42.57688,1.66769,AD,602,Europe/Andorra,2017-11-06
1,El Tarter,42.57952,1.65362,AD,1052,Europe/Andorra,2012-11-03
2,Sant Julià de Lòria,42.46372,1.49129,AD,8022,Europe/Andorra,2013-11-23
3,Pas de la Casa,42.54277,1.73361,AD,2363,Europe/Andorra,2008-06-09
4,Ordino,42.55623,1.53319,AD,3066,Europe/Andorra,2018-10-26
...,...,...,...,...,...,...,...
196586,Beitbridge,-22.21667,30.00000,ZW,26459,Africa/Harare,2013-03-12
196587,Beatrice,-18.25283,30.84730,ZW,1647,Africa/Harare,2018-05-09
196588,Banket,-17.38333,30.40000,ZW,9641,Africa/Harare,2013-03-12
196589,Epworth,-17.89000,31.14750,ZW,123250,Africa/Harare,2012-01-19


In [20]:
matching_cities = np.intersect1d(df['location_of_formation'].dropna().values,cities['cityName'].values)
matching_cities = pd.Series(matching_cities).rename('matching cities')
matching_cities

0                   A Coruña
1                  A Estrada
2       A Pobra do Caramiñal
3                     Aachen
4                    Aalborg
                ...         
1656                   İzmir
1657                    Łódź
1658                 Šentjur
1659                   Žalec
1660                   Žatec
Name: matching cities, Length: 1661, dtype: object

In [21]:
cities = cities[cities['cityName'].isin(matching_cities)]
cities = cities.sort_values('pop.').drop_duplicates(['cityName','region'], keep='last')
cities = cities[['countryCode','cityName','region','lat','lon','pop.','updatedLast']]
cities = cities.sort_values(['countryCode','cityName']).reset_index(drop=True)
cities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3264 entries, 0 to 3263
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   countryCode  3264 non-null   object 
 1   cityName     3264 non-null   object 
 2   region       3264 non-null   object 
 3   lat          3264 non-null   float64
 4   lon          3264 non-null   float64
 5   pop.         3264 non-null   int64  
 6   updatedLast  3264 non-null   object 
dtypes: float64(2), int64(1), object(4)
memory usage: 178.6+ KB


In [22]:
# country_codes = pd.read_html('https://www.iban.com/country-codes', keep_default_na=False)[0]
# country_codes.to_csv('../warehouse/country_codes')
# country_codes = pd.read_csv('../warehouse/country_codes',usecols=[1,2,3,4], dtype=object, na_filter=False)
# country_codes.info()

In [23]:
# replace_dict = {
#     'Country': {
#         'Congo (the Democratic Republic of the)': 'Democratic Republic of the Congo',
#         'Congo (the)': 'Rupublic of the Congo',
#         "Korea (the Democratic People's Republic of)": 'North Korea',
#         'Korea (the Republic of)': 'South Korea'
#     }
# }
# country_codes = country_codes.replace(replace_dict)
# country_codes['Country'] = country_codes['Country'].str.replace(' ?[\(\[](?!British|U\.S\.).*?[\)\]]', '')
# country_codes.info()

In [24]:
mrft_countries = {
    'country_of_origin': {
        'Azerbaijan Soviet Socialist Republic': 'Azerbaijan',
        'Great Britain': 'United Kingdom',
        "Ivory Coast": "Côte d'Ivoire",
        'Kingdom of the Netherlands': 'Netherlands',
        "People's Republic of China": "China",
        'Socialist Federal Republic of Yugoslavia': 'Yugoslavia',
        'Kingdom of Albania': 'Albania',
        'Czech Republic': 'Czechia',
        'Sapin': 'Spain',
    },
    'Country': {
        'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
        'Republic of North Macedonia': 'North Macedonia',
        'Russian Federation': 'Russia',
        'Cabo Verde': 'Cape Verde'
    }
}

In [25]:
df = df.replace(mrft_countries)
# country_codes = country_codes.replace(mrft_countries)

# country_codes.info()

In [26]:
# country_codes.to_csv('../warehouse/country_codes_reformatted')
country_codes = pd.read_csv(
    '../warehouse/country_codes_reformatted',
    usecols=[1,2,3,4],
    dtype=object,
    na_filter=False
).drop(['Alpha-3 code','Numeric'],axis=1)

country_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Country       249 non-null    object
 1   Alpha-2 code  249 non-null    object
dtypes: object(2)
memory usage: 4.0+ KB


In [27]:
other_places = pd.DataFrame([['Czechoslovakia','CZSK'],
                                ['England','GB'],
                                ['German Democratic Republic','DEDR'],
                                ['Northern Ireland','GB'],
                                ['Russian Soviet Federative Socialist Republic','RUFR'],
                                ['Scotland','GB'],
                                ['Soviet Union','USSR'],
                                ['Sápmi','SCND'],
                                ['Wales','GB'],
                                ['West Germany','DEWG'],
                                ['Yugoslavia','YUGO']],
                               columns=['Country','Alpha-2 code'])
country_codes = pd.concat([country_codes,other_places],ignore_index=True)

In [28]:
countries_mismatch = np.setdiff1d(df['country_of_origin'].dropna().values, country_codes['Country'].values)
print(countries_mismatch, ' Length:', len(countries_mismatch))

['Basque Country' 'Brittany' 'California' 'Catalonia' 'Greensboro'
 'Guaynabo' 'Missouri' 'New York' 'Okinawa Prefecture' 'Tuva Republic']  Length: 10


In [29]:
mask = (df['location_of_formation'].isna())&(df['country_of_origin'].isin(countries_mismatch))

df.loc[mask,'location_of_formation'] = df.loc[mask,'country_of_origin'].copy()

In [30]:
df = df.replace({
    'country_of_origin':{
        'Basque Country': 'Spain',
        'California': 'United States of America',
        'Catalonia': 'Spain',
        'Greensboro': 'United States of America',
        'Guaynabo': 'Puerto Rico',
        'Missouri': 'United States of America',
        'New York': 'United States of America',
        'Okinawa Prefecture': 'Japan',
        'Tuva Republic': 'Russia',
        'Brittany': 'France'
    }
})

In [31]:
def series_setdiff(s1,s2):
    diff = np.setdiff1d(s1.dropna().values, s2.dropna().values)
    print(diff, ' Length:', len(diff))

def series_intersect(s1,s2):
    intersection = np.intersect1d(s1.dropna().values, s2.dropna().values)
    print(intersection, ' Length:', len(intersection))

In [33]:
series_setdiff(cities['countryCode'],country_codes['Alpha-2 code'])

[]  Length: 0


In [34]:
series_intersect(cities['countryCode'],country_codes['Alpha-2 code'])

['AD' 'AF' 'AG' 'AM' 'AO' 'AR' 'AT' 'AU' 'AW' 'AX' 'AZ' 'BA' 'BE' 'BG'
 'BM' 'BO' 'BR' 'BY' 'BZ' 'CA' 'CH' 'CL' 'CN' 'CO' 'CR' 'CU' 'CV' 'CY'
 'CZ' 'DE' 'DK' 'DM' 'DO' 'DZ' 'EC' 'EE' 'EG' 'ES' 'FI' 'FK' 'FO' 'FR'
 'GB' 'GD' 'GI' 'GM' 'GR' 'GT' 'GY' 'HN' 'HR' 'HT' 'HU' 'ID' 'IE' 'IL'
 'IN' 'IR' 'IS' 'IT' 'JM' 'JP' 'KE' 'KG' 'KN' 'KR' 'KZ' 'LB' 'LI' 'LR'
 'LS' 'LT' 'LU' 'LV' 'MA' 'MG' 'MK' 'ML' 'MS' 'MT' 'MU' 'MX' 'MY' 'MZ'
 'NF' 'NG' 'NI' 'NL' 'NO' 'NP' 'NZ' 'PA' 'PE' 'PH' 'PK' 'PL' 'PR' 'PT'
 'PY' 'RO' 'RS' 'RU' 'SA' 'SC' 'SE' 'SG' 'SI' 'SK' 'SL' 'SM' 'SN' 'SR'
 'SV' 'TH' 'TR' 'TW' 'UA' 'US' 'UY' 'VE' 'ZA' 'ZW']  Length: 122


In [35]:
len(country_codes)

260

In [36]:
mask = df['location_of_formation']==df['country_of_origin']

df.loc[mask,'location_of_formation'] = np.NaN

In [37]:
df[mask]

,artist,genre,location_of_formation,country_of_origin
28,0.8Syooogeki,j-punk,NaN,Japan
164,2 Times Terror,finnish metal,NaN,Finland
205,3 Doors Down,rap metal,NaN,United States of America
208,3 Doors Down,pop rock,NaN,United States of America
211,3 Doors Down,alternative metal,NaN,United States of America
...,...,...,...,...
112021,Yarabi,house music,NaN,Romania
112236,Yolanda Be Cool,electro swing,NaN,Australia
112364,Yüksek Sadakat,rock music,NaN,Turkey
112460,Zebda,middle of the road,NaN,France


In [38]:
df = df.rename(columns={'country_of_origin': 'Country'})

In [39]:
df = pd.merge(df,country_codes,on='Country')

In [40]:
df = df.drop_duplicates()

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57013 entries, 0 to 57019
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   artist                 57013 non-null  object
 1   genre                  57013 non-null  object
 2   location_of_formation  32718 non-null  object
 3   Country                57013 non-null  object
 4   Alpha-2 code           57013 non-null  object
dtypes: object(5)
memory usage: 2.6+ MB


In [42]:
df[df['location_of_formation'].notna()].nunique()

artist                   7493
genre                    1615
location_of_formation    1884
Country                    95
Alpha-2 code               95
dtype: int64

In [43]:
df[df['location_of_formation'].isna()].nunique()

artist                   7662
genre                    1707
location_of_formation       0
Country                   109
Alpha-2 code              109
dtype: int64

In [44]:
df[df['location_of_formation'].notna()].artist.unique()

array(['!!!', "'68 Comeback", '+44', ..., 'Waylander', 'The Shadows',
       "Winter's Verge"], dtype=object)

In [45]:
series_setdiff(df['Alpha-2 code'],cities['countryCode'])

['AL' 'BD' 'BF' 'CD' 'CI' 'CZSK' 'DEDR' 'DEWG' 'GE' 'GH' 'GN' 'GW' 'HK'
 'KH' 'MD' 'ME' 'MM' 'MN' 'MV' 'NE' 'RUFR' 'SCND' 'TN' 'TT' 'UKCM' 'UKEN'
 'UKNI' 'UKSC' 'USSR' 'YUGO']  Length: 30


In [46]:
series_intersect(df['location_of_formation'],cities['cityName'])

['A Coruña' 'A Estrada' 'A Pobra do Caramiñal' ... 'Łódź' 'Šentjur'
 'Žalec']  Length: 1557


In [47]:
left_merge_col = df['location_of_formation']+df['Alpha-2 code']
right_merge_col = cities['cityName']+cities['countryCode']
df = pd.merge(df,cities,left_on=left_merge_col,right_on=right_merge_col).drop(['key_0','Alpha-2 code','region'], axis=1)

In [48]:
mask = df['genre'].str.contains('rock')
df.loc[mask,'genre'].unique().size

178

In [1]:
df.info()

NameError: name 'df' is not defined

In [50]:
m = folium.Map(location=[51, 0.3],
              zoom_start=5)

for la, lo, ar, ge, ci in df[['lat','lon','artist','genre','cityName']].iloc[::len(df)//500].values:
    folium.Marker(
        location=[la,lo],
        tooltip=f'{ar}<br/>{ge}<br/>{ci}',
        icon=folium.Icon(icon_color='white')
    ).add_to(m)

m

In [51]:
homes_of = []
for genre in df['genre'].unique():
    city = df[df['genre']==genre]['location_of_formation'].value_counts().index[0]
    count = df[df['genre']==genre]['location_of_formation'].value_counts().iloc[0]
    la, lo = cities[cities['cityName']==city][['lat','lon']].values[0]
    homes_of.append(tuple([la, lo, city, genre, count]))

homes_of

[(-41.07223, 145.90731, 'Brooklyn', 'alternative dance', 28),
 (-33.35709, 149.98148, 'Portland', 'dance-punk', 20),
 (-41.07223, 145.90731, 'Brooklyn', 'new rave', 24),
 (-41.07223, 145.90731, 'Brooklyn', 'electroclash', 8),
 (40.21532, -86.02637, 'Atlanta', 'progressive post-hardcore', 8),
 (-31.85251, 115.97493999999999, 'Dayton', 'screamo', 15),
 (40.21532, -86.02637, 'Atlanta', 'dreamo', 16),
 (40.71427, -74.00596999999999, 'New York City', 'afrobeat', 4),
 (-19.86528, -47.44, 'Sacramento', 'funky breaks', 1),
 (25.52206, -103.56756999999999, 'Los Angeles', 'alternative hip hop', 9),
 (40.71427, -74.00596999999999, 'New York City', 'turntablism', 6),
 (40.21532, -86.02637, 'Atlanta', 'hip hop', 16),
 (39.704209999999996, -86.39944, 'Plainfield', 'deep funk', 6),
 (-33.35709, 149.98148, 'Portland', 'abstract hip hop', 5),
 (25.52206, -103.56756999999999, 'Los Angeles', 'pop rock', 14),
 (-41.07223, 145.90731, 'Brooklyn', 'comic', 4),
 (25.52206, -103.56756999999999, 'Los Angeles', 

In [53]:
m2 = folium.Map(location=[51, 0.3],
              zoom_start=5)

for la, lo, ci, ge, num in homes_of:
    if num > 1:
        folium.Marker(
            location=[la,lo],
            tooltip=f'{ci} is the home of: {ge}<br/>with {num} {ge} band(s)',
            icon=folium.Icon(icon_color='white')
        ).add_to(m2)

m2

In [91]:
df[(df['genre']=='boston rock')]

,artist,genre,location_of_formation,Country,countryCode,cityName,lat,lon,pop.,updatedLast
1518,The Bags,boston rock,Los Angeles,United States of America,US,Los Angeles,34.05223,-118.24368,3971883,2019-12-12
7448,Bad Rabbits,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7460,Belly,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7465,Bent Knee,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7471,Big Dipper,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7480,Black Helicopter,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7483,Blake Babies,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7484,Bon Savants,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7534,Dumptruck,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05
7545,Gentlemen Hall,boston rock,Boston,United States of America,US,Boston,42.35843,-71.05977,667137,2019-09-05


In [100]:
cities[cities['region'].str.contains('/.+/')]

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=3264)] are in the [columns]"

In [70]:
cities['region'] = cities['region'].str.split('/', n=1)

In [99]:
cities['region'].values

array([list(['Europe', 'Andorra']), list(['Asia', 'Kabul']),
       list(['America', 'Antigua']), ...,
       list(['Africa', 'Johannesburg']), list(['Africa', 'Johannesburg']),
       list(['Africa', 'Harare'])], dtype=object)

In [89]:
cities[cities['cityName']=='Wales']

,countryCode,cityName,region,lat,lon,pop.,updatedLast,continent
1124,GB,Wales,"[Europe, London]",53.34061,-1.28162,5956,2011-07-31,NaN
3136,US,Wales,"[America, New_York]",42.06954,-72.22230,1767,2017-05-23,NaN
3137,US,Wales,"[America, Chicago]",43.00445,-88.37676,2573,2017-05-23,NaN


In [80]:
len(cities)

3264

In [87]:
(cities['countryCode']=='US').sum()/3264

0.4270833333333333